In [1]:
library(tidyverse)
library(rvest)
library(stringr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




In [45]:
# Get department links
department_links <- read_html("https://centralsquare.dailytable.org/departments") %>%
    html_nodes(".widget-item") %>%
    html_attr("href")

In [47]:
# Get category links
category_links <- list()
for (department_link in department_links) {
    base_url <- "https://centralsquare.dailytable.org"
    category_links <- c(
        category_links,
        read_html(paste0(base_url, department_link)) %>%
            html_nodes("a") %>%
            html_attr("href") %>%
            str_subset("/store/daily-table-central-square/category/") %>%
            unique()
    )
}

In [54]:
get_prices <- function(page) {
    prices <- page %>%
        html_nodes("p.h5") %>%
        html_text() %>%
        str_squish()

    names <- page %>%
        html_nodes("h3.product-title-card") %>%
        html_text() %>%
        str_squish()

    return(tibble("Item"=names, "Price"=prices) %>%
        separate(Price, c("Price", "Units"), " / ")
        )
}

price_list <- tibble()

for (category_link in category_links) {
    # Get first page
    page1 <- read_html(paste0(base_url, category_link, "?page=", 1))

    # Determine total number of items
    category_item_count <- page1 %>%
        html_nodes(".text-grey") %>%
        html_text() %>%
        str_subset("Found") %>%
        str_extract("[0-9]+") %>%
        as.numeric()

    # Get items the first page
    price_list <- price_list %>% bind_rows(get_prices(page1))

    # Get items from subsequent pages
    page_count <- ceiling(category_item_count / 24)
    for (iter_page in 2:page_count) {
        page <- read_html(paste0(base_url, category_link, "?page=", iter_page))
        price_list <- price_list %>% bind_rows(get_prices(page))
    }
}


In [56]:
price_list %>%
    write_csv("daily_table_prices.csv")

price_list

Item,Price,Units
<chr>,<chr>,<chr>
Baby Carrots,$ 1.29,each
Baby Spinach,$ 1.99,lb
Beets,$ 0.69,lb
Broccoli,$ 1.29,lb
Brussels Sprouts,$ 1.49,lb
Butternut Squash,$ 0.79,lb
Carrots,$ 0.69,lb
Cauliflower,$ 3.29,each
Celery Bunch,$ 1.49,each
